#    DOCEREE  - Machine Learning Hackathon (TechGig)

In [22]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LogisticRegression
plt.style.use('bmh')

In [2]:
test       = pd.read_csv(r"E:\Chrome Download\Doceree\Doceree-HCP_Test.csv")
train      = pd.read_csv(r"E:\Chrome Download\Doceree\Doceree-HCP_Train.csv")
submission = pd.read_csv(r"E:\Chrome Download\Doceree\Doceree-HCP_Sample_Submission.csv")

In [3]:
train.head(3)

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0


In [4]:
test.head(3)

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...


In [5]:
submission.head()

,ID,IS_HCP
0,115501,1
1,115502,1
2,115503,1
3,115504,1
4,115505,1


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113937 non-null  int64  
 1   DEVICETYPE       113937 non-null  object 
 2   PLATFORM_ID      113937 non-null  int64  
 3   BIDREQUESTIP     113937 non-null  object 
 4   USERPLATFORMUID  113933 non-null  object 
 5   USERCITY         107578 non-null  object 
 6   USERZIPCODE      109345 non-null  float64
 7   USERAGENT        113935 non-null  object 
 8   PLATFORMTYPE     113937 non-null  object 
 9   CHANNELTYPE      113937 non-null  object 
 10  URL              113937 non-null  object 
 11  KEYWORDS         113937 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(10)
memory usage: 12.2+ MB


In [7]:
train.describe()

,ID,PLATFORM_ID,USERZIPCODE,IS_HCP
count,113937.000000,113937.000000,1.093450e+05,113936.000000
mean,57969.000000,3.574993,1.830240e+07,0.299993
std,32890.923148,2.569019,1.066908e+08,0.458257
min,1001.000000,1.000000,6.020000e+02,0.000000
25%,29485.000000,2.000000,1.935500e+04,0.000000
50%,57969.000000,2.000000,4.321000e+04,0.000000
75%,86453.000000,6.000000,7.709000e+04,1.000000
max,114937.000000,16.000000,9.970149e+08,1.000000


In [8]:
train.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [9]:
data = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142430 entries, 0 to 113936
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               142430 non-null  int64  
 1   DEVICETYPE       142430 non-null  object 
 2   PLATFORM_ID      142430 non-null  int64  
 3   BIDREQUESTIP     142430 non-null  object 
 4   USERPLATFORMUID  142426 non-null  object 
 5   USERCITY         134512 non-null  object 
 6   USERZIPCODE      136723 non-null  float64
 7   USERAGENT        142427 non-null  object 
 8   PLATFORMTYPE     142430 non-null  object 
 9   CHANNELTYPE      142430 non-null  object 
 10  URL              142430 non-null  object 
 11  KEYWORDS         142430 non-null  object 
 12  ind              142430 non-null  object 
 13  TAXONOMY         32313 non-null   object 
 14  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(11)
memory usage: 17.4+ MB


In [11]:
data['DEVICETYPE'].value_counts(dropna=False)

Desktop    98154
Mobile     40011
Tablet      4235
Unknown       30
Name: DEVICETYPE, dtype: int64

In [12]:
data['PLATFORM_ID'].value_counts(dropna=False)

2     98857
7     26188
9      4860
5      4030
6      3740
10     1319
4       945
14      897
12      688
3       607
8       159
1        47
15       47
13       45
16        1
Name: PLATFORM_ID, dtype: int64

In [13]:
data['BIDREQUESTIP'].value_counts(dropna=False)

71.105.120.171     2139
142.154.205.202    1435
68.132.98.211      1084
108.175.114.100     875
108.175.114.101     870
                   ... 
204.186.55.231        1
66.115.66.194         1
24.28.32.41           1
174.193.81.35         1
65.60.230.64          1
Name: BIDREQUESTIP, Length: 37059, dtype: int64

In [14]:
data['USERPLATFORMUID'].value_counts(dropna=False)

3e2578c8-f794-41af-a38c-c5cfb3c0f014    1807
d76f7c0b-1a64-4d4b-b36d-86c832be8837    1750
d00f28d6-3a50-46cd-92f3-8601bd57ad0e     844
fcd3d327-0ad5-425e-8ea8-33ffe6a31543     766
fe53f32c-4646-4abe-afea-90b21d6b8531     763
                                        ... 
4b7a381d-98e0-48a7-bc49-a8bd0da0cb91       1
50f8f49e-5825-45cf-8741-3b48f78c2d9c       1
4ba0fe09-aa1c-43cb-90b8-38864d98e9b3       1
f6852418-67e0-483e-976c-d5aaf0198108       1
356ecd5e-e457-4ffe-8e94-52ac0e5f9c04       1
Name: USERPLATFORMUID, Length: 63426, dtype: int64

In [15]:
data['USERCITY'].value_counts(dropna=False)

NaN                7918
New York           6147
Brooklyn           4038
St Louis           3210
Los Angeles        2729
                   ... 
Charlton              1
West Memphis          1
Long Pond             1
South Deerfield       1
Calumet City          1
Name: USERCITY, Length: 4548, dtype: int64

In [16]:
data['PLATFORMTYPE'].value_counts(dropna=False)

Online Medical Journal      137653
Online Learning Portal        3740
Medical News Platform          989
Electronic Health Record        47
Tele-medicine Platform           1
Name: PLATFORMTYPE, dtype: int64

In [17]:
data['CHANNELTYPE'].value_counts(dropna=False)

Website    142430
Name: CHANNELTYPE, dtype: int64

In [18]:
data['TAXONOMY'].value_counts(dropna=False)     

NaN           110117
2084P0800X      7930
2084N0400X      6621
207Q00000X      3997
207R00000X      3190
               ...  
2085P0229X         1
202C00000X         1
1835P2201X         1
163WC0400X         1
207PP0204X         1
Name: TAXONOMY, Length: 208, dtype: int64

In [30]:
# select_col  =  ['ID','DEVICETYPE','PLATFORM_ID','PLATFORMTYPE']
select_col  =  ['ID','PLATFORM_ID']

In [33]:
train['PLATFORM_ID'] = train['PLATFORM_ID'].astype('int64')

In [39]:
X = train[select_col]
y = train['IS_HCP']

In [40]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2, random_state=0)
print('Total no. of samples: Training and Testing dataset separately!')
print('X_train:', np.shape(X_train))
print('y_train:', np.shape(y_train))
print('X_test:', np.shape(X_test))
print('y_test:', np.shape(y_test))

Total no. of samples: Training and Testing dataset separately!
X_train: (91149, 2)
y_train: (91149,)
X_test: (22788, 2)
y_test: (22788,)
